Gene Expression Data Preprocessing

Overview:
This notebook performs normalization, filtering, and statistical analysis of gene expression data
from single-cell RNA sequencing (scRNA-seq).

The workflow involves:
1. Loading and cleaning the gene expression data from a CSV file.
2. Normalizing the data by calculating the gene expression ratio for each gene in each cell.
   (Each cell's gene expression value is divided by the total RNA detected for that cell.)
3. Filtering out low-expression genes, removing genes expressed in less than 1% of the cells.
4. Computing key statistics for each gene, including the sum of gene expression, mean expression ratio,
   and variance.
5. Saving the results to a csv file for further analysis.

In [1]:
# Required Libraries
import pandas as pd
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor

In [2]:
# Function to load and clean data in chunks, allowing for large files to be processed
def load_and_clean_data(file_path, chunksize=1000):
    """
    Loads and cleans the data from a CSV file in chunks.
    It removes rows where all values are zero.
    
    Parameters:
    - file_path: str, the path to the CSV file
    - chunksize: int, the number of rows per chunk

    Returns:
    - data: DataFrame, the concatenated and cleaned data
    """
    chunks = []  # List to store chunks
    total_rows = sum(1 for _ in open(file_path)) - 1  # Calculate total rows minus header row
    
    # Read the data in chunks and remove rows where all values are zero
    try:
        with pd.read_csv(file_path, index_col=0, chunksize=chunksize) as reader:
            for chunk in tqdm(reader, desc="Reading and cleaning data", total=total_rows // chunksize):
                # Remove rows where all values are zero
                chunk = chunk.loc[~(chunk == 0).all(axis=1)]
                chunks.append(chunk)
    except pd.errors.ParserError as e:
        print(f"Error reading CSV file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    
    # Concatenate the chunks into a single DataFrame
    data = pd.concat(chunks, axis=0)
    return data

In [3]:
# Function to normalize the gene expression data by dividing each gene expression by the column sum
def normalize_gene_expression(data):
    """
    Normalizes gene expression data by dividing each value by the sum of its column (i.e., total expression per gene).
    
    Parameters:
    - data: DataFrame, the gene expression data
    
    Returns:
    - normalized_data: DataFrame, the normalized data
    """
    column_sums = data.sum(axis=0)  # Sum of expression values for each gene
    normalized_data = data.copy()   # Create a copy to avoid modifying the original data
    
    # Normalize each column by dividing by its column sum
    for col in tqdm(data.columns, desc="Normalizing gene expression"):
        normalized_data[col] = data[col] / column_sums[col]
    
    return normalized_data

In [4]:
# Function to filter out genes with low expression based on a threshold
def filter_low_expression_genes(data, threshold=0.01):
    """
    Filters out genes with low expression across cells based on a threshold.
    
    Parameters:
    - data: DataFrame, the normalized gene expression data
    - threshold: float, the minimum percentage of cells where a gene must be expressed to retain the gene
    
    Returns:
    - filtered_data: DataFrame, the filtered data
    """
    num_cells = data.shape[1]  # Number of cells (columns)
    min_cells_expressed = threshold * num_cells  # Minimum number of cells required for a gene to be expressed
    non_zero_counts = (data > 0).sum(axis=1)  # Count of non-zero values per gene (row)
    
    # Use .loc to filter rows (genes) based on the condition
    filtered_data = data.loc[non_zero_counts >= min_cells_expressed]
    
    # Print the number of genes retained after filtering
    print(f"Filtering complete: {filtered_data.shape[0]} genes retained out of {data.shape[0]} total.")
    
    return filtered_data

In [5]:
# Function to compute gene statistics like sum, mean, and variance for each gene
def compute_gene_statistics(data):
    """
    Computes gene statistics such as sum, mean, and variance for each gene across all cells.
    
    Parameters:
    - data: DataFrame, the filtered gene expression data
    
    Returns:
    - gene_statistics: dict, a dictionary of gene statistics where keys are gene names
      and values are lists containing sum, mean, and variance for each gene
    """
    gene_statistics = {}  # Dictionary to store statistics
    
    # Function to calculate statistics for a single gene
    def calc_stats(gene):
        gene_values = data.loc[gene].values  # Get expression values for the gene
        gene_sum = gene_values.sum()         # Total expression across all cells
        mean_expression = gene_values.mean() # Mean expression value
        variance = gene_values.var()         # Variance of expression
        return gene, [gene_sum, mean_expression, variance]

    # Use tqdm to show progress bar while calculating gene statistics
    with ThreadPoolExecutor() as executor:
        results = tqdm(executor.map(calc_stats, data.index), total=len(data.index), desc="Calculating gene statistics")
        
        # Update the dictionary with calculated statistics
        for gene, stats in results:
            gene_statistics[gene] = stats

    return gene_statistics

In [6]:
# Function to save gene statistics to a CSV file
def save_dict_to_csv(gene_statistics, output_file="gene_statistics.csv"):
    """
    Saves the gene statistics dictionary to a CSV file.
    
    Parameters:
    - gene_statistics: dict, the dictionary containing gene statistics
    - output_file: str, the name of the output CSV file
    
    Returns:
    - None
    """
    import csv
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        # Write header row
        writer.writerow(["Gene", "Sum", "Mean", "Variance"])
        # Write gene statistics
        for gene, stats in gene_statistics.items():
            writer.writerow([gene] + stats)
    
    print(f"Gene statistics saved to '{output_file}'.")

In [7]:
def process_gene_expression_data(file_path, threshold=0.01, chunksize=1000):
    """
    Processes the gene expression data by loading, cleaning, normalizing, filtering,
    and computing statistics for gene expression data.
    
    Parameters:
    - file_path: str, the path to the gene expression data file
    - threshold: float, the minimum percentage of cells where a gene must be expressed
    - chunksize: int, the number of rows to read per chunk when loading data
    
    Returns:
    - gene_statistics: dict, the dictionary of computed gene statistics
    """
    # Load and clean the data in chunks
    data = load_and_clean_data(file_path, chunksize=chunksize)
    
    # Normalize the gene expression data
    print("Normalizing gene expression data...")
    normalized_data = normalize_gene_expression(data)
    
    # Filter out low-expression genes based on the threshold
    print("Filtering low-expression genes...")
    filtered_data = filter_low_expression_genes(normalized_data, threshold)
    
    # Compute gene statistics for the filtered data
    gene_statistics = compute_gene_statistics(filtered_data)
    
    return gene_statistics

In [8]:
# Input Path Prompt for Reproducibility
# The user is prompted to input the path to the CSV file dynamically
file_path = input("Please enter the path to your gene expression CSV file (dense matrix): ")
threshold = 0.01  # Threshold for filtering low-expression genes
chunksize = 1000  # Number of rows to read per chunk

In [9]:
# Process the data and save results
try:
    # Process the gene expression data and compute statistics
    gene_statistics = process_gene_expression_data(file_path, threshold, chunksize)

    # Save the computed gene statistics to a CSV file
    save_dict_to_csv(gene_statistics, "gene_statistics.csv")

except Exception as e:
    print(f"An error occurred during processing: {e}")

Reading and cleaning data: 61it [02:10,  2.14s/it]                        


Normalizing gene expression data...


Normalizing gene expression: 100%|██████████| 12333/12333 [00:20<00:00, 601.93it/s]


Filtering low-expression genes...
Filtering complete: 19603 genes retained out of 43011 total.


Calculating gene statistics: 100%|██████████| 19603/19603 [07:33<00:00, 43.25it/s]  


Gene statistics saved to 'gene_statistics.csv'.
